In [ ]:
!pip install pinecone-client==3.2.2 sentence-transformers==2.7.0 datasets==2.19.0 faiss-cpu==1.8.0 transformers==4.40.1 openai==1.25.2 llama-index==0.10.34 llama-index-vector-stores-pinecone==0.1.6  -qqq

# **실습 데이터 다운로드**

In [ ]:
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xf sift.tar.gz
!mkdir data/sift1M -p
!mv sift/* data/sift1M

# **실습 데이터 불러오기**

In [ ]:
import psutil

def get_memory_usage_mb():
  # 현재 실행 중인 프로세스 정보 가져오기
  process = psutil.Process()
  # 프로세스의 메모리 사용량 정보 가져오기
  memory_info = process.memory_info()

  # memory_info.rss -> Resident Set Size(RSS) -> 프로세스가 실제로 물리적 메모리에 로드된 크기
  # 바이트를 MB로 변환
  return memory_info.rss / (1024 * 1024)

In [ ]:
import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

# DatasetSIFT1M -> FAISS에서 제공하는 SIFT1M 데이터셋을 로드하는 클래스
# SIFT1M -> 100만 개(1M) SIFT(Scale-Invariant Feature Transform) 벡터로 구성된 데이터셋
# 이미지 검색, 유사도 검색 등의 벤치마킹에 많이 사용됨
ds = DatasetSIFT1M()

# 검색에 사용할 데이터
xq = ds.get_queries()
# 저장된 벡터 데이터
xb = ds.get_database()
# 질문에 대한 실제 정답 데이터
gt = ds.get_groundtruth()

# **데이터가 늘어날 때 색인/검색 시간, 메모리 사용량 변화**

In [ ]:
k = 1
# 벡터 차원 추출 (128)
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# i를 2씩 증가
for i in range(1, 10, 2):
  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index = faiss.IndexFlatL2(d)
  index.add(xb[:(i + 1) * 100000])

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  print(f"데이터 {(i + 1) * 100000}개:")
  print(f"색인: {(end_indexing - start_indexing) * 1000 :.3f} ms ({end_memory - start_memory:.3f} MB) 검색: {(t1 - t0) * 1000 / nq :.3f} ms")